In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.sparse.linalg
from scipy import optimize
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
import yaml
import timeit
from assignment13 import OneDimReservoir

In [150]:
class PermeabilityEstimator(BaseEstimator, RegressorMixin, OneDimReservoir):
    
    def __init__(self, inputs, subestimator = None):
        
        #stores input dictionary as class attribute, either read from a yaml
        #file or directly from a Python dictonary
        if isinstance(inputs, str):
            with open(inputs) as f:
                self.inputs = yaml.load(f, yaml.FullLoader)
        else:
            self.inputs = inputs
        
        #Initiate OneDimReservoir and obtain initial guess
        self.reservoir = OneDimReservoir(self.inputs)
        self.k_array_length = len(self.reservoir.k)
        self.subestimator = subestimator
        
        return
    
    def run_res_sim(self, k):
        #Runs through OneDimReservoir to obtain final pressure values
        
        #Change permeability values

        self.reservoir.k = k 
        
        #Reapply initial conditions
        
        self.reservoir.apply_initial_conditions()
        
        #Solve for pressures with new permeabilities
        
        self.reservoir.fill_matrices()
        
        self.reservoir.solve()
        
        return self.reservoir.get_solution()
    
    def fit(self, k, history):
        
        #Initiate function
        run_res_sim = self.run_res_sim
        
        #Fine real permeability
        self.coef_ = optimize.newton_krylov(
            lambda k:(run_res_sim(k) - history)**2, k)
                
        return self    
    
    def predict(self, k):
        
        return self.run_res_sim(k)

In [175]:
# Solution = OneDimReservoir('Hetero(100-200)mD.yml')
# Solution.solve()
# SolutionPressure = Solution.get_solution()
# print(SolutionPressure)
# k = np.ones(50) * 150

In [174]:
# l = PermeabilityEstimator('Hetero(100-200)mD.yml')
# l.fit(k, SolutionPressure)
# NewSolution = l.predict(l.coef_)
# print(l.score(k, GuessPressure))
# print(l.score(k, SolutionPressure))
# print(l.score(l.coef_, SolutionPressure))
# # for i in range(len(k)):
# #     error = abs(l.coef_[i]-80)
# #     print(error)
# # print(sum(abs(l.coef_-80)))
# print(l.coef_)

In [173]:
# Guess = OneDimReservoir('100mD.yml')
# Guess.solve()
# GuessPressure = Guess.get_solution()
# print(GuessPressure)

In [171]:
# print(SolutionPressure)
# print(NewSolution)
# print(GuessPressure)

In [172]:
# plt.plot(NewSolution, SolutionPressure)

In [11]:
# match = SolvePermeabilityNewtonKrylov('80mD2.0.yml', SolutionPressure)
# %prun match.optimize_k()